# Modeling GNNePCSAFT

## Starting point

In [ ]:
#    import os
#    import torch
#    os.environ['TORCH'] = torch.__version__
#    print(torch.__version__)

#   !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
%pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
%pip install rdkit torchmetrics ml-collections polars clu contextlib2 -q --no-deps
!mkdir pcsaft
!cp -a /kaggle/input/pcsaftpkg/PC-SAFT-1.4.1/. /kaggle/working/pcsaft
%pip install ./pcsaft -q
!rm -rf pcsaft
%pip install ogb dvc-s3 -q

!pip show pcsaft | grep -E "Version|Name"

!lscpu | grep -E "Model name|Core|core" 

!nvidia-smi

In [ ]:
import os, shutil
import wandb

from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("awskeyid")
secret_value_1 = user_secrets.get_secret("awssecretkey")
secret_value_2 = user_secrets.get_secret("ghtk")
secret_value_3 = user_secrets.get_secret("kaggle")
secret_value_4 = user_secrets.get_secret("username")
secret_value_5 = user_secrets.get_secret("wandbkey")

os.environ["KAGGLE_USERNAME"] = secret_value_4
os.environ["KAGGLE_KEY"] = secret_value_3
os.environ["WANDB_API_KEY"] = secret_value_5
os.environ["AWS_ACCESS_KEY_ID"] = secret_value_0
os.environ["AWS_SECRET_ACCESS_KEY"] = secret_value_1
wandb.login()

In [ ]:
# for the cloud
!rm -rf *
!git clone --branch experiments --single-branch -q https://{secret_value_2}@github.com/wildsonbbl/gnnepcsaft.git .

#!mkdir gnnepcsaft/train/checkpoints
#!cp /kaggle/input/gnnepcsaft/* /kaggle/working/gnnepcsaft/train/checkpoints/

In [ ]:
!dvc pull

## Training

In [ ]:
from gnnepcsaft.configs.default import get_config
config = get_config()
dataset = config.dataset
model_name = config.model_name
checkpoint = config.checkpoint
model_suffix = str(config.num_train_steps / 1e6).replace(".", "_")

In [ ]:
if checkpoint:
    shutil.copy(f"gnnepcsaft/train/checkpoints/{checkpoint}", "gnnepcsaft/train/checkpoints/last_checkpoint.pth")

In [ ]:
!python -m gnnepcsaft.train.train --workdir=./gnnepcsaft --config=gnnepcsaft/configs/default.py --dataset={dataset}

### Updating kaggle dataset

In [ ]:
#!kaggle datasets metadata -p /kaggle/working/gnnepcsaft/train/checkpoints wildsonbblima/gnnepcsaft

shutil.copy("gnnepcsaft/train/checkpoints/last_checkpoint.pth", f"gnnepcsaft/train/checkpoints/{model_name}-{model_suffix}e6.pth")

os.remove("gnnepcsaft/train/checkpoints/last_checkpoint.pth")

#!kaggle datasets version -p /kaggle/working/gnnepcsaft/train/checkpoints -m f"{model_name} at {model_suffix}e6 steps"
!dvc push
print("finished")

### Clear workspace

In [ ]:
!rm -rf * .*